<p><span>In general, each&nbsp;</span><strong>earnings season begins</strong><span>&nbsp;one or two weeks after the last month of</span></p>
<p><span> each quarter (December, March, June, and September).</span></p>
<p><span>look for the majority of public companies to release their&nbsp;</span><strong>earnings</strong><span>&nbsp;in early to mid-January, April, July, and October.</span></p>

In [1]:
#Get the iWidget stuff
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import os
import sys

sys.path.append('/home/michael/jupyter/local-packages')
import numpy as np
import pandas as pd
import datetime
import random
import itertools

# Get my Utilities (localUtilities)
from localUtilities import dateUtils
# from localUtilities.webScrape import earningYahoo
from localUtilities.ibUtils import getOptionStrategyPrice as strat

from localUtilities.webScrape import getEarningOnDate

# Doc is here: https://ib-insync.readthedocs.io/readme.html
from ib_insync import *
util.startLoop()

ib = IB()
# Connect to IB Gateway / Paper Trade 4002 
#            TWS        / Paper Trade 7495
ib.connect('127.0.0.1', 4002, clientId=9)

<IB connected to 127.0.0.1:4002 clientId=9>

In [2]:
aDay = '2019-08-16'
earningsDF = getEarningOnDate.getEarningsOnDate(aDay)

1-32 of 32 results
Pages:  1


In [3]:
anExchange = 'SMART'
theCurrency = 'USD'

# add info 
earningsDF['histVolatility']=np.nan
earningsDF['impliedVolatility']=np.nan
earningsDF['avOptionVolume']=np.nan
earningsDF['close']=np.nan
earningsDF['Expected_Range']=np.nan

In [4]:
earningsDF

,Symbol,Earnings_Date,Company,Earnings Call Time,histVolatility,impliedVolatility,avOptionVolume,close,Expected_Range
0,NAT,2019-08-16,Nordic American Tanker Ltd,Before Market Open,NaN,NaN,NaN,NaN,NaN
1,ATRX,2019-08-16,Adhera Therapeutics Inc,After Market Close,NaN,NaN,NaN,NaN,NaN
2,VJET,2019-08-16,Voxeljet AG,TAS,NaN,NaN,NaN,NaN,NaN
3,AHIX,2019-08-16,ALUF Holdings Inc,Time Not Supplied,NaN,NaN,NaN,NaN,NaN
4,JG,2019-08-16,Aurora Mobile Ltd,Before Market Open,NaN,NaN,NaN,NaN,NaN
5,HMNY,2019-08-16,Helios and Matheson Analytics Inc,Before Market Open,NaN,NaN,NaN,NaN,NaN
6,BLIAQ,2019-08-16,BB Liquidating Inc,Time Not Supplied,NaN,NaN,NaN,NaN,NaN
7,BEBE,2019-08-16,bebe stores inc,After Market Close,NaN,NaN,NaN,NaN,NaN
8,BLIBQ,2019-08-16,BB Liquidating Inc,Time Not Supplied,NaN,NaN,NaN,NaN,NaN
9,XBOR,2019-08-16,Cross Border Resources Inc,Time Not Supplied,NaN,NaN,NaN,NaN,NaN


In [5]:

for row in earningsDF.itertuples():
    print(row.Symbol, end=", ")

    contract = Stock(row.Symbol, anExchange, theCurrency )
    qualityContracts = contract 
    ib.qualifyContracts(qualityContracts)
    qualityContracts
        
    mktData =  ib.reqMktData(contract, "100,101,104,105, 106", False, False, [])
    
    ib.sleep(3)
    
    aticker = ib.ticker(contract)

    earningsDF.at[row.Index, 'histVolatility']= mktData.histVolatility
    earningsDF.at[row.Index, 'impliedVolatility']=mktData.impliedVolatility
    earningsDF.at[row.Index, 'avOptionVolume']=mktData.avOptionVolume
    earningsDF.at[row.Index, 'close']=mktData.last
    earningsDF.at[row.Index, 'Expected_Range']=strat.getExpectedPriceRangeTillNextExpiryDays(mktData.close, mktData.impliedVolatility)

    ib.sleep(2)
    ib.cancelMktData(mktData.contract)


NAT, ATRX, VJET, AHIX, JG, HMNY, BLIAQ, BEBE, BLIBQ, XBOR, DHCPQ, DHCPQ, 

Error 200, reqId 27: No security definition has been found for the request, contract: Stock(symbol='HYCT', exchange='SMART', currency='USD')
Unknown contract: Stock(symbol='HYCT', exchange='SMART', currency='USD')
Error 200, reqId 28: No security definition has been found for the request, contract: Stock(symbol='HYCT', exchange='SMART', currency='USD')


HYCT, 

Error 300, reqId 28: Can't find EId with tickerId:28
Error 200, reqId 29: No security definition has been found for the request, contract: Stock(symbol='PHIOW', exchange='SMART', currency='USD')
Unknown contract: Stock(symbol='PHIOW', exchange='SMART', currency='USD')
Error 200, reqId 30: No security definition has been found for the request, contract: Stock(symbol='PHIOW', exchange='SMART', currency='USD')


PHIOW, 

...

DHCPW, 

Error 300, reqId 40: Can't find EId with tickerId:40
Error 200, reqId 41: No security definition has been found for the request, contract: Stock(symbol='DHCPW', exchange='SMART', currency='USD')
Unknown contract: Stock(symbol='DHCPW', exchange='SMART', currency='USD')
Error 200, reqId 42: No security definition has been found for the request, contract: Stock(symbol='DHCPW', exchange='SMART', currency='USD')


DHCPW, 

Error 300, reqId 42: Can't find EId with tickerId:42


XIN, DE, TTEN, RASF, FLXI, RASFN, GNPG, GHG, RBCN, QD, WFTIQ, NPK, 

In [6]:
theDirectory = '/home/michael/jupyter/earningDateData/'
csvName = '.csv'

file = theDirectory + aDay + csvName

earningsDF.to_csv(file)




In [7]:
earningsDF

,Symbol,Earnings_Date,Company,Earnings Call Time,histVolatility,impliedVolatility,avOptionVolume,close,Expected_Range
0,NAT,2019-08-16,Nordic American Tanker Ltd,Before Market Open,0.375715,0.663492,552.0,1.8850,0.174579
1,ATRX,2019-08-16,Adhera Therapeutics Inc,After Market Close,1.578611,NaN,NaN,0.1600,NaN
2,VJET,2019-08-16,Voxeljet AG,TAS,0.418257,1.215242,13.0,1.7700,0.296212
3,AHIX,2019-08-16,ALUF Holdings Inc,Time Not Supplied,3.606713,NaN,NaN,0.0003,NaN
4,JG,2019-08-16,Aurora Mobile Ltd,Before Market Open,0.742585,NaN,NaN,4.9300,NaN
5,HMNY,2019-08-16,Helios and Matheson Analytics Inc,Before Market Open,1.991851,NaN,3.0,0.0018,NaN
6,BLIAQ,2019-08-16,BB Liquidating Inc,Time Not Supplied,5.803812,NaN,NaN,0.0025,NaN
7,BEBE,2019-08-16,bebe stores inc,After Market Close,0.451333,NaN,NaN,NaN,NaN
8,BLIBQ,2019-08-16,BB Liquidating Inc,Time Not Supplied,NaN,NaN,NaN,NaN,NaN
9,XBOR,2019-08-16,Cross Border Resources Inc,Time Not Supplied,NaN,NaN,NaN,0.0250,NaN
